## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-16-51-58 +0000,nypost,Rothschilds put $537M stake in The Economist u...,https://nypost.com/2025/10/08/media/rothschild...
1,2025-10-08-16-51-17 +0000,48hills,Drama Masks: ‘Hot Wing King’ pours on subversi...,https://48hills.org/2025/10/drama-masks-hot-wi...
2,2025-10-08-16-47-55 +0000,nyt,"In Food Label Fight, Europeans Debate How the ...",https://www.nytimes.com/2025/10/08/business/eu...
3,2025-10-08-16-47-47 +0000,startribune,Temple Israel in Minneapolis tagged with antis...,https://www.startribune.com/temple-israel-in-m...
4,2025-10-08-16-47-44 +0000,nypost,Illegal parking still rampant in NYC neighborh...,https://nypost.com/2025/10/08/us-news/illegal-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
203,trump,36
135,new,18
111,gaza,14
88,shutdown,12
87,government,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-10-08-14-46-38 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...,89
336,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,86
27,2025-10-08-16-01-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...,85
191,2025-10-08-06-58-07 +0000,cbc,National Guard troops are outside Chicago and ...,https://www.cbc.ca/news/world/national-guard-c...,80
240,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...,79


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,89,2025-10-08-14-46-38 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...
86,56,2025-10-08-13-33-06 +0000,nypost,Ex-FBI Director James Comey insists he’s innoc...,https://nypost.com/2025/10/08/us-news/ex-fbi-d...
55,53,2025-10-08-14-44-00 +0000,wsj,"In Japan, as in the U.S., a new leader wants t...",https://www.wsj.com/economy/central-banking/go...
301,52,2025-10-07-20-45-00 +0000,wsj,White House Says Federal Workers’ Back Pay Dur...,https://www.wsj.com/politics/policy/customurl-...
336,50,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
325,36,2025-10-07-19-20-11 +0000,nypost,"Tesla unveils long-awaited cheaper Model Y, Mo...",https://nypost.com/2025/10/07/business/tesla-u...
230,35,2025-10-08-00-44-44 +0000,nyt,Athens Democracy Forum: As Democracy Erodes Gl...,https://www.nytimes.com/2025/10/07/world/europ...
327,34,2025-10-07-19-00-53 +0000,bbc,Watch: Key moments from Pam Bondi's tense Sena...,https://www.bbc.com/news/videos/cpvle9pv1k0o?a...
109,34,2025-10-08-12-06-52 +0000,nypost,California gubernatorial hopeful Katie Porter ...,https://nypost.com/2025/10/08/us-news/californ...
83,34,2025-10-08-13-51-28 +0000,nypost,Inside the Hunter Biden-linked proposal to sel...,https://nypost.com/2025/10/08/us-news/hunter-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
